# Import Libraries

In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" or "tensorflow" or "torch" 

import keras_cv
import keras
import keras.backend as K
import tensorflow as tf
import tensorflow_io as tfio

import numpy as np 
import pandas as pd

from glob import glob
from tqdm import tqdm

import librosa
import IPython.display as ipd
import librosa.display as lid

import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = mpl.cm.get_cmap('coolwarm')

2025-05-23 17:59:45.904054: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-23 17:59:45.904212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-23 17:59:46.110945: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_18/1585041062.py:23: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')


# Configuration

In [2]:
class CFG:
    seed = 42
    
    # Input image size and batch size
    img_size = [128, 384]
    
    # Audio duration, sample rate, and length
    duration = 15 # second
    sample_rate = 32000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    
    # Number of epochs, model name
    preset = 'efficientnetv2_b2_imagenet'

    # Class Labels for BirdCLEF 25
    class_names = sorted(os.listdir('/kaggle/input/birdclef-2025/train_audio/'))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {v:k for k,v in label2name.items()}

# Reproducibility
Sets value for random seed to produce similar result in each run.

In [3]:
tf.keras.utils.set_random_seed(CFG.seed)

# Dataset Path

In [4]:
BASE_PATH = '/kaggle/input/birdclef-2025'

# Test Data

In [5]:
test_paths = glob(f'{BASE_PATH}/test_soundscapes/*ogg')
# During commit use `train_soundscapes` data as there is no `test` data.
# During submission `test_soundscapes` data will automatically be populated.
if len(test_paths)==0:
    test_paths = glob(f'{BASE_PATH}/train_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.head()

,filepath
0,/kaggle/input/birdclef-2025/train_soundscapes/...
1,/kaggle/input/birdclef-2025/train_soundscapes/...
2,/kaggle/input/birdclef-2025/train_soundscapes/...
3,/kaggle/input/birdclef-2025/train_soundscapes/...
4,/kaggle/input/birdclef-2025/train_soundscapes/...


# Modeling

Note that our model was trained on `10 second` duration audio files, but we will infer on `5-second` audio files (as per competition rules). To facilitate this, we have set the model input shape to `(None, None, 3)`, which will allow us to have variable-length input during training and inference.

In [6]:
# Create an input layer for the model
inp = keras.layers.Input(shape=(None, None, 3))
# Pretrained backbone
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
    CFG.preset,
)
out = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=CFG.num_classes,
    name="classifier"
)(inp)
# Build model
model = keras.models.Model(inputs=inp, outputs=out)
# Load weights of trained model
model.load_weights("/kaggle/input/train-birdclef25-kerascv/best_model.weights.h5")

Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...


# Data Loader

The following code will decode the raw audio from `.ogg` file and also decode the spectrogram from the `audio` file. Additionally, we will apply Z-Score standardization and Min-Max normalization to ensure consistent inputs to the model.

In [7]:
# Decodes Audio
def build_decoder(with_labels=True, dim=1024):
    def get_audio(filepath):
        file_bytes = tf.io.read_file(filepath)
        audio = tfio.audio.decode_vorbis(file_bytes) # decode .ogg file
        audio = tf.cast(audio, tf.float32)
        if tf.shape(audio)[1]>1: # stereo -> mono
            audio = audio[...,0:1]
        audio = tf.squeeze(audio, axis=-1)
        return audio
    
    def create_frames(audio, duration=5, sr=32000):
        frame_size = int(duration * sr)
        audio = tf.pad(audio[..., None], [[0, tf.shape(audio)[0] % frame_size], [0, 0]]) # pad the end
        audio = tf.squeeze(audio) # remove extra dimension added for padding
        frames = tf.reshape(audio, [-1, frame_size]) # shape: [num_frames, frame_size]
        return frames
    
    def apply_preproc(spec):
        # Standardize
        mean = tf.math.reduce_mean(spec)
        std = tf.math.reduce_std(spec)
        spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)

        # Normalize using Min-Max
        min_val = tf.math.reduce_min(spec)
        max_val = tf.math.reduce_max(spec)
        spec = tf.where(tf.math.equal(max_val - min_val, 0), spec - min_val,
                              (spec - min_val) / (max_val - min_val))
        return spec

    def decode(path):
        # Load audio file
        audio = get_audio(path)
        # Split audio file into frames with each having 5 seecond duration
        audio = create_frames(audio)
        # Convert audio to spectrogram
        spec = keras.layers.MelSpectrogram(num_mel_bins=CFG.img_size[0],
                                             fft_length=CFG.nfft, 
                                              sequence_stride=CFG.hop_length, 
                                              sampling_rate=CFG.sample_rate)(audio)
        # Apply normalization and standardization
        spec = apply_preproc(spec)
        # Covnert spectrogram to 3 channel image (for imagenet)
        spec = tf.tile(spec[..., None], [1, 1, 1, 3])
        return spec
    
    return decode

In [8]:
# Build data loader
def build_dataset(paths, batch_size=1, decode_fn=None, cache=False):
    if decode_fn is None:
        decode_fn = build_decoder(dim=CFG.audio_len) # decoder
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths,)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO) # decode audio to spectrograms then create frames
    ds = ds.cache() if cache else ds # cache files
    ds = ds.batch(batch_size, drop_remainder=False) # create batches
    ds = ds.prefetch(AUTO)
    return ds

# Inference

In [9]:
# Initialize empty list to store ids
ids = []

# Initialize empty array to store predictions
preds = np.empty(shape=(0, CFG.num_classes), dtype='float32')

# Build test dataset
test_paths = test_df.filepath.tolist()
test_ds = build_dataset(paths=test_paths, batch_size=1)

# Iterate over each audio file in the test dataset
for idx, specs in enumerate(tqdm(iter(test_ds), desc='test ', total=len(test_df))):
    # Extract the filename without the extension
    filename = test_paths[idx].split('/')[-1].replace('.ogg','')
    
    # Convert to backend-specific tensor while excluding extra dimension
    specs = keras.ops.convert_to_tensor(specs[0])
    
    # Predict bird species for all frames in a recording using all trained models
    frame_preds = model.predict(specs, verbose=0)
    
    # Create a ID for each frame in a recording using the filename and frame number
    frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(frame_preds))]
    
    # Concatenate the ids
    ids += frame_ids
    # Concatenate the predictions
    preds = np.concatenate([preds, frame_preds], axis=0)

test : 100%|██████████| 10/10 [00:08<00:00,  1.15it/s]


# Submission

In [10]:
# Submit prediction
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, CFG.class_names] = preds
pred_df.to_csv('submission.csv',index=False)
pred_df.head()

,row_id,1139490,1192948,1194042,126247,1346504,134933,135045,1462711,1462737,...,yebfly1,yebsee1,yecspi2,yectyr1,yehbla2,yehcar1,yelori1,yeofly1,yercac1,ywcpar
0,H27_20230421_155000_5,0.000126,0.000048,0.000036,0.000037,0.000015,0.000019,0.000043,0.000068,0.000088,...,0.001614,0.000416,0.000369,0.000126,0.000157,0.000097,0.003912,0.002047,0.000435,0.000050
1,H27_20230421_155000_10,0.000140,0.000074,0.000036,0.000077,0.000032,0.000062,0.000225,0.000098,0.000154,...,0.008863,0.000797,0.001613,0.001405,0.000716,0.000716,0.003513,0.019489,0.000712,0.000198
2,H27_20230421_155000_15,0.000232,0.000156,0.000052,0.000117,0.000285,0.000215,0.000492,0.000124,0.000438,...,0.013232,0.005964,0.023722,0.003197,0.002669,0.001660,0.001834,0.029799,0.001742,0.000668
3,H27_20230421_155000_20,0.000140,0.000123,0.000041,0.000166,0.000199,0.000426,0.000224,0.000080,0.000171,...,0.015796,0.001916,0.006196,0.007164,0.000723,0.003185,0.000663,0.025879,0.000895,0.000335
4,H27_20230421_155000_25,0.000133,0.000111,0.000041,0.000170,0.000176,0.000404,0.000181,0.000091,0.000169,...,0.013331,0.002066,0.005659,0.007774,0.001838,0.004333,0.001150,0.037410,0.001109,0.000505
